# **Nivel SparkSQL**

**Instalação do PySpark**

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Importação de biliotecas**

In [3]:
# CHAMAR AS BIBLIOTECAS e MÓDULOS NECESSÁRIAS

from pyspark import SparkConf
from google.colab import drive
from google.cloud import storage
from pyspark.sql import SparkSession
import os
import csv
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType, DoubleType

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Configuração de Acesso para os GCP**

In [5]:
# CONFIGURAÇÃO DA CHAVE DE SEGURANÇA

serviceAccount = '/content/weighty-archive-349117-ab0ee1ad5b41.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [6]:
# CÓDIGO QUE ACESSA A BUCKET CRIADA E FAZ O DOWNLOAD DOS ARQUIVOS VIA PANDAS

client = storage.Client()

# CRIAR UMA VARIÁVEL CHAMADA BUCKET QUE VAI RECEBER O NOME DA BUCKET DO CLOUD STORAGE
bucket = client.get_bucket('soulcodemarketing')

# USAR O MÉTODO BLOB PARA RETORNAR O NOME DO ARQUIVO (JSON, CSV, PARQUET)
bucket.blob('marketing_campaign.csv')

 #CRIA UMA VARIÁVEL PATH PARA COLOCAR O CAMINHO DO CSV
path = 'gs://soulcodemarketing/marketing_campaign.csv'

**Conecção com o Apache-Spark**

In [7]:
# CRIAR A SPARK SESSION E LER O ARQUIVO VIA PYSPARK

spark = (SparkSession.builder\
        .master("local")\
        .appName("mkt_dfsql")\
        .config("spark.ui.port", "4050")\
        .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')\
        .getOrCreate())

In [8]:
spark  # Verifica a conecção

**Carrega dados do arquivo - df_pd_marketing_campaign.csv**

In [20]:
df = (spark
      .read
      .format("csv")
      .option("header", "true")
      .option("inferschema", "true")
      .option("delimiter", ",")
      .load('/content/df_pd_marketing_campaign.csv')
      #.load(path)
      .createOrReplaceTempView("marketing"))

**1 - Imprime a tabela na tela 10 linhas.**

In [29]:
spark.sql('''SELECT * FROM marketing''').show()

+---+----+--------------+------------------+-------------+-----------------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+----------------+-------+--------+
|_c0|  id|ano_nascimento|          educacao| estado_civil|      renda_anual|qtd_crianca|qtd_adolescente|primeira_data_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|contato_de_custo|receita|resposta|
+---+----+--------------+------------------+-------------+-----------------+-----------+---------------+------------

**2 - Imprime os indexes da tabela.**

In [17]:
spark.sql('''DESCRIBE marketing''').show()

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|        00|      int|   null|
|      5524|      int|   null|
|      1957|      int|   null|
|  Graduado|   string|   null|
|  Solteiro|   string|   null|
|   58138.0|   double|   null|
|        06|      int|   null|
|        07|      int|   null|
|2012-09-04|   string|   null|
|        58|      int|   null|
|       635|      int|   null|
|      8811|      int|   null|
|       546|      int|   null|
|       172|      int|   null|
|      8814|      int|   null|
|      8815|      int|   null|
|         3|      int|   null|
|         8|      int|   null|
|        10|      int|   null|
|         4|      int|   null|
+----------+---------+-------+
only showing top 20 rows



**3 - Seleção da renda anual e perfil de compras com base nos dados socio-economicos**

In [22]:
spark.sql('''
            SELECT id, renda_anual,  qtd_compras_online, educacao, estado_civil, dias_desde_ult_compras, qtd_visitas_online_mes
            FROM marketing
            ORDER BY renda_anual DESC;
          ''').show()

+-----+-----------+------------------+---------+-------------+----------------------+----------------------+
|   id|renda_anual|qtd_compras_online| educacao| estado_civil|dias_desde_ult_compras|qtd_visitas_online_mes|
+-----+-----------+------------------+---------+-------------+----------------------+----------------------+
| 9432|   666666.0|                 3| Graduado|Uniao_estavel|                    23|                     6|
| 1503|   162397.0|                 0|Doutorado|Uniao_estavel|                    31|                     1|
| 1501|   160803.0|                 0|Doutorado|       Casado|                    21|                     0|
| 5336|   157733.0|                 1| Mestrado|Uniao_estavel|                    37|                     1|
| 8475|   157243.0|                 0|Doutorado|       Casado|                    98|                     0|
| 4931|   157146.0|                 0| Graduado|Uniao_estavel|                    13|                     1|
|11181|   156924.0|

**4 - Estudo com criterio de quantdade de compras, renda anual e dias desde da ultima compra.**

In [23]:
spark.sql('''
            SELECT id, qtd_compras_online, renda_anual, dias_desde_ult_compras
            FROM marketing
            ORDER BY qtd_compras_online DESC;
          ''').show(10)

+-----+------------------+-----------------+----------------------+
|   id|qtd_compras_online|      renda_anual|dias_desde_ult_compras|
+-----+------------------+-----------------+----------------------+
| 5255|                27|52247.25135379061|                    19|
| 4619|                27|         113734.0|                     9|
|10311|                25|           4428.0|                     0|
| 6237|                23|           7144.0|                    92|
| 8931|                11|          83033.0|                    82|
| 7030|                11|          66465.0|                     1|
| 9365|                11|          60000.0|                    57|
| 6439|                11|          61074.0|                    37|
| 4299|                11|          70971.0|                    28|
| 7381|                11|          75693.0|                    10|
+-----+------------------+-----------------+----------------------+
only showing top 10 rows



**5 -  Estudo com critério do maior numero de dias sem comprar.**

In [24]:
spark.sql('''
            SELECT id, dias_desde_ult_compras, qtd_compras_online, renda_anual 
            FROM marketing
            ORDER BY dias_desde_ult_compras DESC;
          ''').show()

+-----+----------------------+------------------+-----------+
|   id|dias_desde_ult_compras|qtd_compras_online|renda_anual|
+-----+----------------------+------------------+-----------+
|  868|                    99|                 2|    44794.0|
| 7829|                    99|                 2|    36640.0|
| 7232|                    99|                 1|    42429.0|
|10142|                    99|                 5|    66476.0|
| 9977|                    99|                 9|    78901.0|
| 8595|                    99|                 1|    42429.0|
| 7947|                    99|                 1|    42231.0|
| 5263|                    99|                 1|    31056.0|
| 3363|                    99|                 1|    20130.0|
| 4070|                    99|                 8|    94871.0|
| 2831|                    99|                 7|    78416.0|
| 2415|                    99|                 5|    62568.0|
| 1743|                    99|                 2|    69719.0|
| 7212| 